#Fast Food SVM
Application of the Fast Food kernel expansion algorithm to SVMs.

Code ported from MATLAB implementation:
 * Ji Zhao, Deyu Meng. FastMMD: Ensemble of Circular Discrepancy for Efficient Two-Sample Test. Neural Computation, 2015. 

In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
#import pylab as pl
#import theano
#import theano.tensor as T
import numpy as np
from numpy.linalg import norm
#import pandas as pd
#import sklearn.datasets
from sklearn import svm
from scipy.linalg import hadamard
from scipy.special import gammaincinv
from functools import partial
from collections import namedtuple

VERBOSE = True  # increases verbosity of some outputs

d = 100 # dimension of input pattern
n = d*20 # basis number used for approximation
sgm = 10 # bandwidth for Gaussian kernel
N = 32 # number of input patterns

# random number generator
rng = np.random.RandomState()

# load data
X1 = rng.normal(size=(d,N))
print('Shape of X1:',X1.shape)
y = rng.choice([0,1],size=N)
print('Shape of y:',y.shape)

# pad with zeros so d is nearest power of 2
# N = X.shape[0]
# l = X.shape[1]
# print('Original shape of X: n =',N,', d =',l)
# d = int(2 ** np.ceil(np.log2(l)))
# if d != l:  # pad only if needed
#     print('Padding input from d =',l,'to d =',d)
#     X = np.pad(X,((0,d-l),(0,0)),mode='constant',constant_values=0)
    
# convert to shared tensors
#X = theano.shared(X,name='X',borrow=True)
#y = theano.shared(y,name='y',borrow=True)

Shape of X1: (100, 32)
Shape of y: (32,)


In [2]:
FFPara = namedtuple('FFPara', 'B G PI S')
def fastfood_params(n,d):
    d0 = d
    n0 = n
    l = int(np.ceil(np.log2(d)))
    d = 2**l
    k = int(np.ceil(n/d))
    n = d*k
#     print('d0 =',d0,', d =',d)
#     print('n0 =',n,', n =',n)
    
    B = []
    G = []
    PI = []
    S = []
    for ii in xrange(k):
        B_ii  = rng.choice([-1,1],size=d)
        G_ii  = rng.normal(size=d)
        PI_ii = rng.permutation(d)
        
        B.append(B_ii)
        G.append(G_ii)
        PI.append(PI_ii)
        
        p1 = rng.uniform(size=d)
        p2 = d/2
#        print('p1 =',p1,'; p2 =',p2)
        T = gammaincinv(p2,p1)
#        print('T1 =',T)
        T = (T*2) ** (1/2)
#        print('T2 =',T)
        s_i = T * norm(G,'fro')**(-1)
#        print('s_i =', s_i)
        S_ii = s_i
        S.append(S_ii)
    
    S1 = np.zeros(n)
    for ii in xrange(k):
        S1[ii*d:(ii+1)*d] = S[ii]
    
#    print('Shape of B:',len(B),', B[0]:',B[0].shape)
    
    return FFPara(B, G, PI, S1)
print('Ready to generate fastfood params')

Ready to generate fastfood params


In [3]:
def bit_reverse_traverse(a):
    # (c) 2014 Ryan Compton
    # ryancompton.net/2014/06/05/bit-reversal-permutation-in-python/
    n = a.shape[0]
    assert(not n&(n-1) ) # assert that n is a power of 2
    if n == 1:
        yield a[0]
    else:
        even_index = np.arange(int(n/2))*2
        odd_index = np.arange(int(n/2))*2 + 1
        for even in bit_reverse_traverse(a[even_index]):
            yield even
        for odd in bit_reverse_traverse(a[odd_index]):
            yield odd

def get_bit_reversed_list(l):
    # (c) 2014 Ryan Compton
    # ryancompton.net/2014/06/05/bit-reversal-permutation-in-python/
    n = len(l)
#    print('n=',n)
    indexs = np.arange(n)
    b = []
    for i in bit_reverse_traverse(indexs):
        b.append(l[i])
    return b

def FWHT(X):
    # Fast Walsh-Hadamard Transform for 1D signals
    # of length n=2^M only (non error-proof for now)
    x=get_bit_reversed_list(X)
    x=np.array(x)
    N=len(X)
 
    for i in range(0,N,2):
        x[i]=x[i]+x[i+1]
        x[i+1]=x[i]-2*x[i+1]
 
    L=1
    y=np.zeros_like(x)
    for n in range(2,int(np.log2(N))+1):
        M=2**L
        J=0; K=0
        while(K<N):
            for j in range(J,J+M,2):
                y[K]   = x[j]   + x[j+M]
                y[K+1] = x[j]   - x[j+M]
                y[K+2] = x[j+1] + x[j+1+M]
                y[K+3] = x[j+1] - x[j+1+M]
                K=K+4
            J=J+2*M
        x=y.copy()
        L=L+1
 
    y=x/float(N)
    
    return y

def fastfood_forkernel(X,para,sgm,use_spiral=False):
    d0, m = X.shape
#    print('d0 =',d0,', m =',m)
    l = int(np.ceil(np.log2(d0)))
    d = 2**l
    if d == d0:
        XX = X
    else:
        XX = np.zeros((d,m))
        XX[0:d0,:] = X
#    print('Shape of XX:',XX.shape)
    
    k = len(para.B)
    n = d*k
    tht = np.zeros((n,m))
    for ii in xrange(k):
        B = para.B[ii]
        G = para.G[ii]
        PI = para.PI[ii]
#        XX = np.dot(B,XX)
        XX = (B * XX.T).T
        T = FWHT(XX)
        T = T[PI,:]
#        print('T.shape:',T.shape,',(G*d).shape:',(G*d).shape)
        T = (G*d) * T.T
#        print('T.shape:',T.shape)
        T = FWHT(T)
#        print('T.shape:',T.shape)
        idx1 = ii*d
        idx2=(ii+1)*d
#        print('idx1=',idx1,',idx2=',idx2)
        tht[idx1:idx2,:] = T.T
        
    S = para.S
    tht = (S*np.sqrt(d) * tht.T).T
    
    T = tht/sgm
    phi = np.concatenate([np.cos(T),np.sin(T)],axis=1)
    phi = 1.0/np.sqrt(n) * phi
    return phi,tht

In [4]:
def ff_kernel(para,sgm,x1,x2):
    # TODO use the precomputed Gram matrix rather than calculating K(x,y) individually
    X = np.hstack((x1,x2))
    phi, tht = fastfood_forkernel(X,para,sgm)
    K_appro = np.dot(phi[0].T,phi[1])
    return K_appro

In [5]:
para = fastfood_params(n,d)
#print('Fastfood params:',params)

print('X1.shape:',X1.shape)
phi1,tht1 = fastfood_forkernel(X1,para,sgm)
print('len(phi1) =',len(phi1),'phi1 =',phi1)
print('len(tht1) =',len(tht1),', tht1 =',tht1)
#K_appro = np.dot(phi1[0].T,phi1[1])
#print('K_appro =',K_appro)

# we create an instance of SVM and fit out data.
my_kernel = lambda x1,x2: ff_kernel(para,sgm,x1,x2)
clf = svm.SVC(kernel=my_kernel)
clf.fit(X1.T,y)


X1.shape: (100, 32)
len(phi1) = 2048 phi1 = [[  7.05261908e-03   2.17331225e-02  -8.51596122e-04 ...,   1.44190182e-02
   -1.33995189e-02  -2.14394186e-02]
 [  1.77706225e-02   1.62938549e-03  -4.83171336e-03 ...,  -1.81884982e-02
    3.15837901e-03   2.13758921e-02]
 [ -1.74252523e-02   1.26544444e-03   8.15230688e-03 ...,  -1.26163318e-03
    3.20572941e-03  -1.65296868e-02]
 ..., 
 [  2.18951606e-02   2.20579326e-02   2.17347369e-02 ...,  -1.28089417e-04
   -6.01131713e-06   4.48465488e-03]
 [  1.35215802e-02   2.20773511e-02   1.30559128e-02 ...,  -1.40846187e-02
    9.48046277e-03  -1.08866426e-02]
 [  2.12485935e-02   1.93784821e-02   2.20207257e-02 ...,   2.52170251e-03
    7.01250423e-03  -6.60475887e-04]]
len(tht1) = 2048 , tht1 = [[ -5.03723737e+01   1.81750175e+00   1.60934471e+01 ...,   7.10918798e+00
   -2.49007611e+01   4.46780161e+01]
 [  6.36457018e+00  -1.49699177e+01  -1.79123570e+01 ...,  -9.66879118e+00
    1.43423119e+00   1.31460637e+01]
 [  2.47928765e+01   1.513

ValueError: operands could not be broadcast together with shapes (128,) (200,32) 